In [1]:
import psycopg2
import sys
import pprint
import pandas
import random

In [2]:
def connect():
    "return a connection"
    conn_string = "host='localhost' dbname='mimic' user='postgres'"
    conn = psycopg2.connect(conn_string)
    return conn
    
def disconnect(conn):
    "disconnect"
    conn.commit()
    conn.close()

def get_data( query, itersize=10000):
    "return a pandas dataframe with columns names"
    conn = connect()
    cur = conn.cursor(f'tmpCursor')
    cur.itersize = itersize
    cur.execute(query)
    pd = pandas.DataFrame(cur.fetchall(),columns= [desc[0] for desc in cur.description])
    cur.close()
    disconnect(conn)
    return pd


In [ ]:
print(get_data("select * from mimiciii.d_items where itemid IN (818, 1531, 22566) "))

In [25]:
 #print(get_data("SELECT table_schema, table_name FROM information_schema.tables WHERE table_schema IN (  'mimiciii')"))

In [26]:
# print(get_data("SELECT * FROM sepsis.general_vars LIMIT 10"))

In [27]:
#ref = get_data("SELECT * FROM mimiciii.d_items WHERE label ~* 'temp' AND linksto = 'chartevents'")
#print(ref)
#ref.to_csv('ITEMS_TEMP_20170606.csv')

In [145]:
def inStmt(col):
    "generates list"
    return ','.join(col.map(str))

In [ ]:
cardiac_index = pandas.read_csv("ITEMS_CARDIAC_INDEX_201511111831.csv")
mean_pressure = pandas.read_csv("ITEMS_MEAN_PRESSURE_201511111723.csv")
systol_pressure = pandas.read_csv("ITEMS_SYSTOL_PRESSURE_201511111657-3.csv")
thromb = pandas.read_csv("ITEMS_THROMB_201511111845.csv")
venus_sat = pandas.read_csv("ITEMS_VENUS_SAT_201511111810.csv")
weight = pandas.read_csv("ITEMS_WEIGHT_20170606.csv")
temp = pandas.read_csv("ITEMS_TEMP_20170606.csv")
tmp = temp[temp['conceptid'] == True]
print(inStmt(tmp['itemid']))
a = get_data("SELECT * FROM mimiciii.d_icd_diagnoses")
a.to_csv('diag.csv')

In [ ]:
print(get_data(f'SELECT max(valuenum), avg(valuenum),  count(1), itemid, valueuom, label \
FROM mimiciii.chartevents \
JOIN mimiciii.d_items USING (itemid) \
WHERE valuenum IS NOT NULL AND itemid IN (225678, 227457, 828, 3789, 6256) \
GROUP BY valueuom, itemid, label order by count DESC;'))

In [ ]:
print(get_data(f'SELECT max(valuenum), avg(valuenum),  count(1), itemid, valueuom, label \
FROM mimiciii.chartevents \
JOIN mimiciii.d_items USING (itemid) \
WHERE valuenum IS NOT NULL AND valuenum < 155 AND  itemid IN (818, 1531, 225668) \
GROUP BY valueuom, itemid, label order by count DESC;'))

In [ ]:
print(get_data("SELECT max(value), avg(value), median(value::numeric), count(1), itemid, label \
        FROM mimiciii.outputevents cd\
        JOIN mimiciii.d_items USING (itemid) \
        WHERE\
        label ~* 'urin'\
        AND  value != 0\
        GROUP BY valueuom, itemid, label order by count DESC;" ))

In [28]:
#print(get_data("SELECT * FROM mimiciii.d_items WHERE label ~* 'urin'"))

In [38]:
ci=cardiac_index[cardiac_index['valid'] == True]
mp=mean_pressure[mean_pressure['valid'] == True]
sp=systol_pressure[systol_pressure['valid'] == True]
th=thromb[thromb['valid'] == True]
vs=venus_sat[venus_sat['valid'] == True]

In [ ]:
print(get_data('SELECT count(1)  FROM sepsis.angus2013_sepsis_severe WHERE hadm_id NOT IN (SELECT hadm_id FROM sepsis.angus2013_sepsis)'))